In [1]:
pwd


'/Users/juka42/Desktop/Udacity/python_AI/project01/workspace'

In [1]:
# Imports only listdir function from OS module 
from os import listdir  

# Retrieve the filenames from folder pet_images/
filename_list = listdir("pet_images/")

# Print 10 of the filenames from folder pet_images/
print("\nPrints 10 filenames from folder pet_images/")
for idx in range(0, len(filename_list), 1):
    print("{:2d} file: {:>25}".format(idx + 1, filename_list[idx]) )


Prints 10 filenames from folder pet_images/
 1 file:                cat_01.jpg
 2 file:          Poodle_07927.jpg
 3 file:                cat_02.jpg
 4 file:      Great_dane_05320.jpg
 5 file:       Dalmatian_04068.jpg
 6 file:              gecko_02.jpg
 7 file:                cat_07.jpg
 8 file:  Great_pyrenees_05435.jpg
 9 file: German_shepherd_dog_04931.jpg
10 file: German_shepherd_dog_04890.jpg
11 file:          Collie_03797.jpg
12 file:   Saint_bernard_08010.jpg
13 file:       Dalmatian_04037.jpg
14 file:            Rabbit_002.jpg
15 file:         polar_bear_04.jpg
16 file:          Poodle_07956.jpg
17 file:       fox_squirrel_01.jpg
18 file:          Beagle_01170.jpg
19 file:  Boston_terrier_02285.jpg
20 file:             skunk_029.jpg
21 file:  Boston_terrier_02303.jpg
22 file: Miniature_schnauzer_06884.jpg
23 file:          Beagle_01141.jpg
24 file:         Basenji_00974.jpg
25 file:              gecko_80.jpg
26 file:       Dalmatian_04017.jpg
27 file:           Boxer_02426.jp

In [2]:
name = filename_list[1].split('.')[0]
print(name)

Poodle_07927


In [3]:
type(filename_list)

list

In [31]:
def labelize(filename):
    label=''
    for i in range(len(filename)):
        if filename[i].isalpha(): 
            label += filename[i].casefold()
        elif  filename[i] == '_':
            label += ' '
        elif filename[i] == '.':
            break
    return(label.strip())

print(labelize(filename_list[39]))

basset hound
   asdf   !
asdf!


In [16]:
labels = {}
file_name = filename_list[1]
label = labelize(file_name.split('.')[0])
labels[file_name]=[label]
print(labels)

{'Poodle_07927.jpg': ['poodle']}


In [81]:
results_dic = {}
for filename in filename_list:
    results_dic[filename] = labelize(filename)
print(results_dic)

{'cat_01.jpg': 'cat', 'Poodle_07927.jpg': 'poodle', 'cat_02.jpg': 'cat', 'Great_dane_05320.jpg': 'great dane', 'Dalmatian_04068.jpg': 'dalmatian', 'gecko_02.jpg': 'gecko', 'cat_07.jpg': 'cat', 'Great_pyrenees_05435.jpg': 'great pyrenees', 'German_shepherd_dog_04931.jpg': 'german shepherd dog', 'German_shepherd_dog_04890.jpg': 'german shepherd dog', 'Collie_03797.jpg': 'collie', 'Saint_bernard_08010.jpg': 'saint bernard', 'Dalmatian_04037.jpg': 'dalmatian', 'Rabbit_002.jpg': 'rabbit', 'polar_bear_04.jpg': 'polar bear', 'Poodle_07956.jpg': 'poodle', 'fox_squirrel_01.jpg': 'fox squirrel', 'Beagle_01170.jpg': 'beagle', 'Boston_terrier_02285.jpg': 'boston terrier', 'skunk_029.jpg': 'skunk', 'Boston_terrier_02303.jpg': 'boston terrier', 'Miniature_schnauzer_06884.jpg': 'miniature schnauzer', 'Beagle_01141.jpg': 'beagle', 'Basenji_00974.jpg': 'basenji', 'gecko_80.jpg': 'gecko', 'Dalmatian_04017.jpg': 'dalmatian', 'Boxer_02426.jpg': 'boxer', 'Basenji_00963.jpg': 'basenji', 'Boston_terrier_0225

In [12]:
results_dic = {}
results_dic["asdf"] = "qwer"
print(results_dic)

{'asdf': 'qwer'}


In [13]:
for idx in range(10):
    print(filename_list[idx])

cat_01.jpg
Poodle_07927.jpg
cat_02.jpg
Great_dane_05320.jpg
Dalmatian_04068.jpg
gecko_02.jpg
cat_07.jpg
Great_pyrenees_05435.jpg
German_shepherd_dog_04931.jpg
German_shepherd_dog_04890.jpg


In [ ]:
for filename in filename_list:
    results_dic[filename] = labelize(filename)

In [42]:
import ast
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.models as models
from torch import __version__

resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

models = {'resnet': resnet18, 'alexnet': alexnet, 'vgg': vgg16}

# obtain ImageNet labels
with open('imagenet1000_clsid_to_human.txt') as imagenet_classes_file:
    imagenet_classes_dict = ast.literal_eval(imagenet_classes_file.read())

def classifier(img_path, model_name):
    # load the image
    img_pil = Image.open(img_path)

    # define transforms
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # preprocess the image
    img_tensor = preprocess(img_pil)
    
    # resize the tensor (add dimension for batch)
    img_tensor.unsqueeze_(0)
    
    # wrap input in variable, wrap input in variable - no longer needed for
    # v 0.4 & higher code changed 04/26/2018 by Jennifer S. to handle PyTorch upgrade
    pytorch_ver = __version__.split('.')
    
    # pytorch versions 0.4 & hihger - Variable depreciated so that it returns
    # a tensor. So to address tensor as output (not wrapper) and to mimic the 
    # affect of setting volatile = True (because we are using pretrained models
    # for inference) we can set requires_gradient to False. Here we just set 
    # requires_grad_ to False on our tensor 
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        img_tensor.requires_grad_(False)
    
    # pytorch versions less than 0.4 - uses Variable because not-depreciated
    else:
        # apply model to input
        # wrap input in variable
        data = Variable(img_tensor, volatile = True) 

    # apply model to input
    model = models[model_name]

    # puts model in evaluation mode
    # instead of (default)training mode
    model = model.eval()
    
    # apply data to model - adjusted based upon version to account for 
    # operating on a Tensor for version 0.4 & higher.
    if int(pytorch_ver[0]) > 0 or int(pytorch_ver[1]) >= 4:
        output = model(img_tensor)

    # pytorch versions less than 0.4
    else:
        # apply data to model
        output = model(data)

    # return index corresponding to predicted class
    pred_idx = output.data.numpy().argmax()

    return imagenet_classes_dict[pred_idx]



In [44]:
classifier("pet_images/German_shepherd_dog_04890.jpg","resnet")

'German shepherd, German shepherd dog, German police dog, alsatian'

In [41]:
!pip install torchvision

     |████████████████████████████████| 387 kB 273 kB/s eta 0:00:01
     |████████████████████████████████| 97.5 MB 10.9 MB/s eta 0:00:01     |█████████████▏                  | 40.0 MB 10.8 MB/s eta 0:00:06


In [129]:
def classify_images(images_dir, results_dic, model):
    """
    Creates classifier labels with classifier function, compares pet labels to 
    the classifier labels, and adds the classifier label and the comparison of 
    the labels to the results dictionary using the extend function. Be sure to
    format the classifier labels so that they will match your pet image labels.
    The format will include putting the classifier labels in all lower case 
    letters and strip the leading and trailing whitespace characters from them.
    For example, the Classifier function returns = 'Maltese dog, Maltese terrier, Maltese' 
    so the classifier label = 'maltese dog, maltese terrier, maltese'.
    Recall that dog names from the classifier function can be a string of dog 
    names separated by commas when a particular breed of dog has multiple dog 
    names associated with that breed. For example, you will find pet images of
    a 'dalmatian'(pet label) and it will match to the classifier label 
    'dalmatian, coach dog, carriage dog' if the classifier function correctly 
    classified the pet images of dalmatians.
     PLEASE NOTE: This function uses the classifier() function defined in 
     classifier.py within this function. The proper use of this function is
     in test_classifier.py Please refer to this program prior to using the 
     classifier() function to classify images within this function 
     Parameters: 
      images_dir - The (full) path to the folder of images that are to be
                   classified by the classifier function (string)
      results_dic - Results Dictionary with 'key' as image filename and 'value'
                    as a List. Where the list will contain the following items: 
                  index 0 = pet image label (string)
                --- where index 1 & index 2 are added by this function ---
                  NEW - index 1 = classifier label (string)
                  NEW - index 2 = 1/0 (int)  where 1 = match between pet image
                    and classifer labels and 0 = no match between labels
      model - Indicates which CNN model architecture will be used by the 
              classifier function to classify the pet images,
              values must be either: resnet alexnet vgg (string)
     Returns:
           None - results_dic is mutable data type so no return needed.         
    """
    acertos = 0
    for key in results_dic:
        result = classifier(images_dir+key,model).casefold()
        if result.find(results_dic[key]) >= 0:
            match = 1
            acertos += 1
        else: match = 0
        results_dic[key] = [results_dic[key], result.casefold(), match]
    print("########### numero de acertos >>> {}".format(acertos))
    None 

In [167]:
results_dic = {}
for filename in filename_list:
    results_dic[filename] = labelize(filename)
print(results_dic["German_shepherd_dog_04931.jpg"])

german shepherd dog


In [168]:
classify_images("pet_images/", results_dic, "alexnet")

########### numero de acertos >>> 30


In [120]:
print(results_dic)

{'cat_01.jpg': ['cat', 'norwegian elkhound, elkhound', 0], 'Poodle_07927.jpg': ['poodle', 'standard poodle, poodle', 1], 'cat_02.jpg': ['cat', 'tabby, tabby cat, cat', 1], 'Great_dane_05320.jpg': ['great dane', 'great dane', 1], 'Dalmatian_04068.jpg': ['dalmatian', 'dalmatian, coach dog, carriage dog', 1], 'gecko_02.jpg': ['gecko', 'african chameleon, chamaeleo chamaeleon', 0], 'cat_07.jpg': ['cat', 'egyptian cat, cat', 1], 'Great_pyrenees_05435.jpg': ['great pyrenees', 'great pyrenees', 1], 'German_shepherd_dog_04931.jpg': ['german shepherd dog', 'german shepherd, german shepherd dog, german police dog, alsatian', 1], 'German_shepherd_dog_04890.jpg': ['german shepherd dog', 'german shepherd, german shepherd dog, german police dog, alsatian', 1], 'Collie_03797.jpg': ['collie', 'collie', 1], 'Saint_bernard_08010.jpg': ['saint bernard', 'saint bernard, st bernard', 1], 'Dalmatian_04037.jpg': ['dalmatian', 'dalmatian, coach dog, carriage dog', 1], 'Rabbit_002.jpg': ['rabbit', 'wood rabbit

In [126]:
acertos = 0
for key in results_dic:
    acertos = acertos + results_dic[key][2]
    if results_dic[key][2] == 1: print(key)
print(acertos)


Poodle_07927.jpg
cat_02.jpg
Great_dane_05320.jpg
Dalmatian_04068.jpg
cat_07.jpg
Great_pyrenees_05435.jpg
German_shepherd_dog_04931.jpg
German_shepherd_dog_04890.jpg
Collie_03797.jpg
Saint_bernard_08010.jpg
Dalmatian_04037.jpg
Rabbit_002.jpg
polar_bear_04.jpg
Poodle_07956.jpg
fox_squirrel_01.jpg
Boston_terrier_02285.jpg
skunk_029.jpg
Boston_terrier_02303.jpg
Miniature_schnauzer_06884.jpg
Beagle_01141.jpg
Basenji_00974.jpg
Dalmatian_04017.jpg
Boxer_02426.jpg
Basenji_00963.jpg
Boston_terrier_02259.jpg
Golden_retriever_05223.jpg
Saint_bernard_08036.jpg
Golden_retriever_05195.jpg
Beagle_01125.jpg
German_shorthaired_pointer_04986.jpg
Cocker_spaniel_03750.jpg
Golden_retriever_05257.jpg
Basset_hound_01034.jpg
33


In [123]:
results_dic['cat_01.jpg']

['cat', 'norwegian elkhound, elkhound', 0]

In [132]:
acertos = 0
for key in results_dic:
    acertos = acertos + results_dic[key][2]
    if results_dic[key][2] != 1: print(key + " - " + results_dic[key][1])
print(acertos)

cat_01.jpg - norwegian elkhound, elkhound
gecko_02.jpg - african chameleon, chamaeleo chamaeleon
Beagle_01170.jpg - walker hound, walker foxhound
gecko_80.jpg - tailed frog, bell toad, ribbed toad, tailed toad, ascaphus trui
Golden_retriever_05182.jpg - leonberg
great_horned_owl_02.jpg - great grey owl, great gray owl, strix nebulosa
Great_pyrenees_05367.jpg - kuvasz
33


In [64]:
print(results_dic)

{'cat_01.jpg': ['cat', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'Poodle_07927.jpg': ['poodle', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'cat_02.jpg': ['cat', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'Great_dane_05320.jpg': ['great dane', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'Dalmatian_04068.jpg': ['dalmatian', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'gecko_02.jpg': ['gecko', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'cat_07.jpg': ['cat', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'Great_pyrenees_05435.jpg': ['great pyrenees', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'German_shepherd_dog_04931.jpg': ['german shepherd dog', 'German shepherd, German shepherd dog, German police dog, alsatian', 0], 'German_shepherd_dog_04890.jpg': ['germ

In [75]:
list = ["asDf, qwEr, asdf"]
asdf = "asdf"
if asdf in list: print("Verdadeiro")
else: print("Falso")

Falso


In [73]:
print(casefold(list))

NameError: name 'casefold' is not defined

In [169]:
dognames = {}
with open('dognames.txt', 'r') as f:
    for line in f:
        dognames[line[:-1]]=1

for key in results_dic:
    if results_dic[key][0] in dognames: 
        results_dic[key].append(1)
    else:
        results_dic[key].append(0)

    if results_dic[key][1] in dognames: 
        results_dic[key].append(1)
    else:
        results_dic[key].append(0)


print(results_dic)

{'cat_01.jpg': ['cat', 'african hunting dog, hyena dog, cape hunting dog, lycaon pictus', 0, 0, 0], 'Poodle_07927.jpg': ['poodle', 'standard poodle, poodle', 1, 1, 1], 'cat_02.jpg': ['cat', 'tiger cat, cat', 1, 0, 0], 'Great_dane_05320.jpg': ['great dane', 'great dane', 1, 1, 1], 'Dalmatian_04068.jpg': ['dalmatian', 'dalmatian, coach dog, carriage dog', 1, 1, 1], 'gecko_02.jpg': ['gecko', 'alligator lizard', 0, 0, 0], 'cat_07.jpg': ['cat', 'egyptian cat, cat', 1, 0, 0], 'Great_pyrenees_05435.jpg': ['great pyrenees', 'great pyrenees', 1, 1, 1], 'German_shepherd_dog_04931.jpg': ['german shepherd dog', 'german shepherd, german shepherd dog, german police dog, alsatian', 1, 1, 1], 'German_shepherd_dog_04890.jpg': ['german shepherd dog', 'german shepherd, german shepherd dog, german police dog, alsatian', 1, 1, 1], 'Collie_03797.jpg': ['collie', 'collie', 1, 1, 1], 'Saint_bernard_08010.jpg': ['saint bernard', 'saint bernard, st bernard', 1, 1, 1], 'Dalmatian_04037.jpg': ['dalmatian', 'dalma

In [142]:
dognames = file_data.split('\n')
print(dognames)

['chihuahua', 'japanese spaniel', 'maltese dog, maltese terrier, maltese', 'pekinese, pekingese, peke', 'shih-tzu', 'blenheim spaniel', 'papillon', 'toy terrier', 'rhodesian ridgeback', 'afghan hound, afghan', 'basset, basset hound', 'beagle', 'bloodhound, sleuthhound', 'bluetick', 'black-and-tan coonhound', 'walker hound, walker foxhound', 'english foxhound', 'redbone', 'borzoi, russian wolfhound', 'irish wolfhound', 'italian greyhound', 'whippet', 'ibizan hound, ibizan podenco', 'norwegian elkhound, elkhound', 'otterhound, otter hound', 'saluki, gazelle hound', 'scottish deerhound, deerhound', 'weimaraner', 'staffordshire bullterrier, staffordshire bull terrier', 'american staffordshire terrier, staffordshire terrier, american pit bull terrier, pit bull terrier', 'bedlington terrier', 'border terrier', 'kerry blue terrier', 'irish terrier', 'norfolk terrier', 'norwich terrier', 'yorkshire terrier', 'wire-haired fox terrier', 'lakeland terrier', 'sealyham terrier, sealyham', 'airedale

In [156]:
results_dic['Dalmatian_04068.jpg']

['dalmatian', 'dalmatian, coach dog, carriage dog', 1]

In [157]:
lista_tst = ['dalmatian', 'dalmatian, coach dog, carriage dog', 1]

In [158]:
lista_tst[2]

1

In [159]:
lista_tst.append(1)
lista_tst

['dalmatian', 'dalmatian, coach dog, carriage dog', 1, 1]

In [163]:
results_dic['Dalmatian_04068.jpg'][1] in dognames

'dalmatian, coach dog, carriage dog'

In [165]:
for key in dognames:
    if results_dic['Dalmatian_04068.jpg'][1] == key:
        print(key)

dalmatian, coach dog, carriage dog


In [166]:
print(dognames)

{'chihuahua': 1, 'japanese spaniel': 1, 'maltese dog, maltese terrier, maltese': 1, 'pekinese, pekingese, peke': 1, 'shih-tzu': 1, 'blenheim spaniel': 1, 'papillon': 1, 'toy terrier': 1, 'rhodesian ridgeback': 1, 'afghan hound, afghan': 1, 'basset, basset hound': 1, 'beagle': 1, 'bloodhound, sleuthhound': 1, 'bluetick': 1, 'black-and-tan coonhound': 1, 'walker hound, walker foxhound': 1, 'english foxhound': 1, 'redbone': 1, 'borzoi, russian wolfhound': 1, 'irish wolfhound': 1, 'italian greyhound': 1, 'whippet': 1, 'ibizan hound, ibizan podenco': 1, 'norwegian elkhound, elkhound': 1, 'otterhound, otter hound': 1, 'saluki, gazelle hound': 1, 'scottish deerhound, deerhound': 1, 'weimaraner': 1, 'staffordshire bullterrier, staffordshire bull terrier': 1, 'american staffordshire terrier, staffordshire terrier, american pit bull terrier, pit bull terrier': 1, 'bedlington terrier': 1, 'border terrier': 1, 'kerry blue terrier': 1, 'irish terrier': 1, 'norfolk terrier': 1, 'norwich terrier': 1,